In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
#print(PROJECT_ROOT)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from model import *

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [2]:
bot = PsychologistRAG(faiss_path="../faiss_index")

In [3]:
# проверяем, что бот работает
question = input("Вопрос: ").strip()
print(question)
bot.ask(question)

{'title': 'Как справиться с ощущением безысходной энергетической истощенности и эмоциональным выгоранием',
 'solution': 'Ваше состояние понятно и важно воспринимать его всерьёз. Вот несколько шагов, которые могут помочь:\n\n1. **Обратитесь за поддержкой**: Не оставайтесь один на один с проблемой. Поделитесь своими переживаниями с близкими людьми, друзьями или коллегами. Важно, чтобы вас выслушали без осуждения и советов — просто дайте себе возможность проговорить всё, что накопилось.\n\n2. **Практикуйте телесный контакт и дыхательные техники**: Если рядом есть человек, которому вы доверяете, попросите его обнять вас или положить руку на плечо. Это помогает снизить уровень тревоги. Попробуйте вместе с ним нормализовать дыхание: медленный вдох, небольшая задержка, плавный выдох. Если под рукой есть пакет, можно подышать в него, чтобы восстановить баланс углекислого газа.\n\n3. **Используйте простые успокаивающие методы**: Помассируйте пальцы рук, мочки ушей, шею или плечи — это помогает 

In [4]:
k = 3

results = []

# проверяем, что бот выбирает статьи с наиболее подходящей тематикой
basic_questions = {
    "depression": "Психиатр поставил мне диагноз 'депрессия', что делать?",
    "anxiety": "Я постоянно испытываю сильную тревогу",
    "burnout": "Я выгорела на работе",
    "panic_attack": "Как справляться с паническими атаками?",
    "stress": "Я испытываюсь сильный стресс"
}

for expected_theme, question in basic_questions.items():
    docs = bot.db.similarity_search(question, k=k)

    retrieved_categories = [
        doc.metadata.get("category") for doc in docs
    ]

    # Hit@k
    hit = int(expected_theme in retrieved_categories)

    # Precision@k
    precision = retrieved_categories.count(expected_theme) / k

    # Reciprocal Rank
    rr = 0
    for i, cat in enumerate(retrieved_categories):
        if cat == expected_theme:
            rr = 1 / (i + 1)
            break

    results.append({
        "question": question,
        "expected_theme": expected_theme,
        "retrieved_categories": retrieved_categories,
        "hit@k": hit,
        "precision@k": precision,
        "rr": rr
    })

results

[{'question': "Психиатр поставил мне диагноз 'депрессия', что делать?",
  'expected_theme': 'depression',
  'retrieved_categories': ['depression', 'depression', 'depression'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Я постоянно испытываю сильную тревогу',
  'expected_theme': 'anxiety',
  'retrieved_categories': ['anxiety', 'paper', 'paper'],
  'hit@k': 1,
  'precision@k': 0.3333333333333333,
  'rr': 1.0},
 {'question': 'Я выгорела на работе',
  'expected_theme': 'burnout',
  'retrieved_categories': ['burnout', 'burnout', 'burnout'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Как справляться с паническими атаками?',
  'expected_theme': 'panic_attack',
  'retrieved_categories': ['panic_attack', 'panic_attack', 'panic_attack'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Я испытываюсь сильный стресс',
  'expected_theme': 'stress',
  'retrieved_categories': ['stress', 'stress', 'paper'],
  'hit@k': 1,
  'precision@k': 0.66

In [5]:
# Считаем метрики на бенчмарке из размеченных 100 вопросов. Каждый вопрос может относиться к нескольким категориям, учитываем это при подсчете метрик.
from eval.eval_retr import evaluate_retrieval

BENCHMARK_PATH = f"{PROJECT_ROOT}/eval/psychrag_bench_100.json"
K = 3  # top-k retrieval

results, _ = evaluate_retrieval(bot, BENCHMARK_PATH, K)

print("\n=== OVERALL METRICS ===")
for k, v in results["overall"].items():
    print(f"{k}: {v:.3f}")

print("\n=== PER-THEME METRICS ===")
for theme, stats in results["per_theme"].items():
    print(f"\n[{theme}]")
    for k, v in stats.items():
        print(f"  {k}: {v:.3f}")



=== OVERALL METRICS ===
Hit@3: 0.930
Precision@3: 0.760
MRR@3: 0.858

=== PER-THEME METRICS ===

[depression]
  Hit@3: 1.000
  Precision@3: 0.985
  MRR@3: 1.000

[anxiety]
  Hit@3: 0.800
  Precision@3: 0.427
  MRR@3: 0.580

[panic_attack]
  Hit@3: 1.000
  Precision@3: 0.905
  MRR@3: 0.964

[stress]
  Hit@3: 1.000
  Precision@3: 0.818
  MRR@3: 1.000

[burnout]
  Hit@3: 0.929
  Precision@3: 0.857
  MRR@3: 0.929

[paper]
  Hit@3: 0.929
  Precision@3: 0.714
  MRR@3: 0.845
